# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
clean_city_data=pd.read_csv('../Resources/Clean City Data.csv')
del clean_city_data['Unnamed: 0']

clean_city_data.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Mokhsogollokh,61,128.9203,28,97,100,3.53,RU,1619732188
1,Busselton,-33,115.3333,48,98,72,9.10,AU,1619732189
2,San Policarpo,12,125.5072,76,84,73,2.48,PH,1619732189
3,Ushuaia,-54,-68.3000,42,56,20,3.44,AR,1619731981
4,Rikitea,-23,-134.9692,76,68,99,8.75,PF,1619732190


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = clean_city_data[["Lat", "Lng"]]

# Fill NaN values and convert to float
weights = clean_city_data["Humidity"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
# Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

ideal_weather=clean_city_data.loc[clean_city_data['Wind Speed']<10]
ideal_weather=ideal_weather.loc[ideal_weather['Max Temp']<90]
ideal_weather=ideal_weather.loc[ideal_weather['Max Temp']>80]
ideal_weather=ideal_weather.loc[ideal_weather['Cloudiness']<10]
ideal_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
130,Sarh,9,18.3923,87,51,0,2.89,TD,1619732244
260,Sokodé,8,1.1333,84,74,0,3.58,TG,1619732301
292,Cayenne,4,-52.3333,81,83,0,2.30,GF,1619732303
338,Natchez,31,-91.4032,86,58,1,5.75,US,1619732167
380,Altamira,-3,-52.2064,84,79,0,3.44,BR,1619732353
389,Saint George,37,-113.5841,84,12,1,8.05,US,1619732357
516,Abu Dhabi,24,54.3667,86,45,0,3.44,AE,1619732413
563,Mehnagar,25,83.1167,85,21,0,6.55,IN,1619732434


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df=ideal_weather

In [15]:
# geocoordinates
target_radius = 5000
target_search = "hotel"
key_search = "hotel"
# set up a parameters dictionary

params = {
#     "location": target_coordinates,
    "radius": target_radius,
    "type": target_search,
    "keyword": key_search,
    "key": g_key}

for index, row in hotel_df.iterrows():
    
    # latitudes and longitudes from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
#     print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
130,Sarh,9,18.3923,87,51,0,2.89,TD,1619732244,NaN
260,Sokodé,8,1.1333,84,74,0,3.58,TG,1619732301,NaN
292,Cayenne,4,-52.3333,81,83,0,2.30,GF,1619732303,NaN
338,Natchez,31,-91.4032,86,58,1,5.75,US,1619732167,Lake Rosemound Inn Bed & Breakfast
380,Altamira,-3,-52.2064,84,79,0,3.44,BR,1619732353,NaN
389,Saint George,37,-113.5841,84,12,1,8.05,US,1619732357,"Hampton Inn & Suites, SunRiver, St. George"
516,Abu Dhabi,24,54.3667,86,45,0,3.44,AE,1619732413,NaN
563,Mehnagar,25,83.1167,85,21,0,6.55,IN,1619732434,NaN


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [25]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))